# CH-05 AI 技術指標回測系統

## 5-2 強大的回測工具：backtesting.py

### 1️⃣ 安裝及匯入套件

In [ ]:
!pip install openai
!pip install yfinance==0.2.38
!pip install backtesting
!pip install bokeh==2.4.3 # 繪圖套件
from  openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import datetime as dt # 時間套件
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


### 2️⃣ 取得股價資料

In [ ]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
df = yf.download(stock_id, period="5y")
# 計算指標
df['ma1'] = df['Close'].rolling(window=5).mean()
df['ma2'] = df['Close'].rolling(window=10).mean()
df.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ma1,ma2
Date,,,,,,,,
2018-11-07,233.0,236.0,232.5,234.0,205.401077,34424652,NaN,NaN
2018-11-08,237.5,237.5,236.0,236.5,207.595581,24739146,NaN,NaN
2018-11-09,234.5,234.5,231.0,231.0,202.767746,23219081,NaN,NaN
2018-11-12,230.0,234.5,230.0,231.5,203.206650,23228445,NaN,NaN
2018-11-13,224.5,228.0,224.5,227.5,199.695480,36104127,232.1,NaN


### 3️⃣ 定義回測策略

In [ ]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
      self.buy(size=1)
    elif crossover(self.data.ma2, self.data.ma1):
      self.sell(size=1)

### 4️⃣ 回測結果

In [ ]:
backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()

# 印出回測績效
print(stats)

# 查看詳細的交易紀錄
stats["_trades"].head()

Start                     2018-11-07 00:00:00
End                       2023-11-07 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   76.853377
Equity Final [$]                   100527.042
Equity Peak [$]                    100666.618
Return [%]                           0.527042
Buy & Hold Return [%]              136.752137
Return (Ann.) [%]                    0.109175
Volatility (Ann.) [%]                0.138633
Sharpe Ratio                         0.787508
Sortino Ratio                        1.209536
Calmar Ratio                         0.465253
Max. Drawdown [%]                   -0.234657
Avg. Drawdown [%]                   -0.032216
Max. Drawdown Duration      617 days 00:00:00
Avg. Drawdown Duration       50 days 00:00:00
# Trades                                   63
Win Rate [%]                        49.206349
Best Trade [%]                      44.828848
Worst Trade [%]                     -13.44591
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,17,24,230.418,220.0,-10.418,-0.045213,2018-11-30,2018-12-11,11 days
1,1,29,76,221.884,231.0,9.116,0.041085,2018-12-18,2019-03-08,80 days
2,1,82,92,240.458,251.0,10.542,0.043841,2019-03-18,2019-04-01,14 days
3,1,93,94,250.498,249.0,-1.498,-0.005980,2019-04-02,2019-04-03,1 days
4,1,95,112,252.004,261.5,9.496,0.037682,2019-04-08,2019-05-02,24 days


### 5️⃣ 回測繪圖

In [ ]:
backtest.plot(plot_equity=True,
       plot_return=False,
       plot_pl=True,
       plot_volume=True,
       plot_drawdown=False,
       superimpose=True)

Row(id='1407', ...)

### 6️⃣ 設定停利、停損點

In [ ]:
class CrossStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.ma1, self.data.ma2):
        # 買入時設置停損與停利價格
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.ma2, self.data.ma1):
        # 賣出時時設置停損與停利價格
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)

backtest = Backtest(df,
        CrossStrategy,
        cash=100000,
        commission=0.004,
        margin=1,
        hedging=False,
        trade_on_close=False,
        exclusive_orders=False,
        )
stats = backtest.run()
print(stats)

Start                     2018-11-07 00:00:00
End                       2023-11-07 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   56.177924
Equity Final [$]                    99901.398
Equity Peak [$]                    100056.974
Return [%]                          -0.098602
Buy & Hold Return [%]              136.752137
Return (Ann.) [%]                   -0.020476
Volatility (Ann.) [%]                0.084842
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.368459
Avg. Drawdown [%]                   -0.052725
Max. Drawdown Duration     1279 days 00:00:00
Avg. Drawdown Duration      148 days 00:00:00
# Trades                                   69
Win Rate [%]                        34.782609
Best Trade [%]                      14.218451
Worst Trade [%]                    -12.260064
Avg. Trade [%]                    

## 5-3 讓 AI 產生回測策略

### 7️⃣ 輸入 OpenAI API KEY

In [ ]:
import getpass
api_key = getpass.getpass("請輸入金鑰：")
client = OpenAI(api_key=api_key)

請輸入金鑰：··········


### 8️⃣ 創建 GPT 3.5 模型函式

In [ ]:
# GPT 3.5 模型
def get_reply(messages):
  try:
    response = client.chat.completions.create(model="gpt-3.5-turbo",
                         messages=messages)
    reply = response.choices[0].message.content
  except OpenAIError as err:
    reply = f"發生 {err.type} 錯誤\n{err.message}"
  return reply

# 設定 AI 角色, 使其依據使用者需求進行 df 處理
def ai_helper(df, user_msg):

  msg = [{
    "role":
    "system",
    "content":
    f"As a professional code generation robot, \
      I require your assistance in generating Python code \
      based on specific user requirements. To proceed, \
      I will provide you with a dataframe (df) that follows the \
      format {df.columns}. Your task is to carefully analyze the \
      user's requirements and generate the Python code \
      accordingly.Please note that your response should solely \
      consist of the code itself, \
      and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg} \n\
      Your task is to develop a Python function named \
      'calculate(df)'. This function should accept a dataframe as \
      its parameter. Ensure that you only utilize the columns \
      present in the dataset, specifically {df.columns}.\
      After processing, the function should return the processed \
      dataframe. Your response should strictly contain the Python \
      code for the 'calculate(df)' function \
      and exclude any unrelated content."
  }]

  reply_data = get_reply(msg)
  return reply_data

# 產生技術指標策略
def ai_strategy(df, user_msg, add_msg="無"):

  code_example ='''
class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.short_ma, self.data.long_ma):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.10)
    elif crossover(self.data.long_ma, self.data.short_ma):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.90)
        '''

  msg = [{
    "role":
    "system",
    "content":
     "As a Python code generation bot, your task is to generate \
     code for a stock strategy based on user requirements and df. \
     Please note that your response should solely \
     consist of the code itself, \
     and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
     "The user requirement:計算 ma,\n\
     The additional requirement: 請設置 10% 的停利與停損點\n\
     The df.columns =['Open',	'High', 'Low',	'Close',	'Adj Close',	'Volume', 'short_ma',	'long_ma']\n\
     Please using the crossover() function in next(self)\
     Your response should strictly contain the Python \
     code for the 'AiStrategy(Strategy)' class \
     and exclude any unrelated content."
  }, {
    "role":
    "assistant",
    "content":f"{code_example}"
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg}\n\
     The additional requirement:{add_msg}\n\
     The df.columns ={df.columns}\n\
     Your task is to develop a Python class named \
     'AiStrategy(Strategy)'\
     Please using the crossover() function in next(self)."

  }]

  reply_data = get_reply(msg)
  return reply_data


### 9️⃣ 計算技術指標

In [ ]:
# 輸入股票代號
stock_id = "2330.tw"
# 抓取 5 年資料
df = yf.download(stock_id, period="5y")
# 計算指標
user_msg = ["MACD", "請設置10%的停損點與20%的停利點"]
code_str = ai_helper(df, user_msg[0])
print(code_str)
exec(code_str)
new_df = calculate(df)
new_df.tail()

[*********************100%%**********************]  1 of 1 completed
def calculate(df):
    # Calculate the MACD line
    df['12-day EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['26-day EMA'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD Line'] = df['12-day EMA'] - df['26-day EMA']
    
    # Calculate the signal line
    df['Signal Line'] = df['MACD Line'].ewm(span=9, adjust=False).mean()
    
    # Calculate the MACD histogram
    df['MACD Histogram'] = df['MACD Line'] - df['Signal Line']
    
    return df


,Open,High,Low,Close,Adj Close,Volume,12-day EMA,26-day EMA,MACD Line,Signal Line,MACD Histogram
Date,,,,,,,,,,,
2023-11-01,533.0,533.0,527.0,528.0,528.0,13603347,536.071721,538.115217,-2.043496,-0.399727,-1.643769
2023-11-02,536.0,547.0,535.0,547.0,547.0,29401438,537.752994,538.773349,-1.020354,-0.523852,-0.496502
2023-11-03,547.0,549.0,545.0,549.0,549.0,15426176,539.483303,539.530878,-0.047575,-0.428597,0.381022
2023-11-06,553.0,556.0,550.0,550.0,550.0,29215201,541.101256,540.306369,0.794887,-0.183900,0.978788
2023-11-07,552.0,555.0,550.0,553.0,553.0,7551711,542.931832,541.246638,1.685194,0.189919,1.495276


### 🔟 策略生成

In [ ]:
strategy_str = ai_strategy(new_df, user_msg[0], user_msg[1])
print(strategy_str)
print("-----------------------")
exec(strategy_str)
backtest = Backtest(df,
        AiStrategy,
        cash=100000,
        commission=0.004,
        trade_on_close=True,
        exclusive_orders=True,
        )
stats = backtest.run()
print(stats)

class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data['MACD Line'], self.data['Signal Line']):
        self.buy(size=1,
                 sl=self.data.Close[-1] * 0.90,
                 tp=self.data.Close[-1] * 1.20)
    elif crossover(self.data['Signal Line'], self.data['MACD Line']):
        self.sell(size=1,
                  sl=self.data.Close[-1] * 1.10,
                  tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2018-11-07 00:00:00
End                       2023-11-07 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   97.034596
Equity Final [$]                   100248.662
Equity Peak [$]                     100352.87
Return [%]                           0.248662
Buy & Hold Return [%]              136.324786
Return (Ann.) [%]                    0.051566
Volatility (Ann.) [%]                0.117311
Sharpe Ratio                         0.439566
Sor

### 1️⃣1️⃣ 寫成函式

In [ ]:
def ai_backtest(stock_id, period, user_msg, add_msg):

  # 下載資料
  df = yf.download(stock_id, period=period)

  # 獲取和執行指標計算程式碼
  code_str = ai_helper(df, user_msg)
  local_namespace = {}
  exec(code_str, globals(), local_namespace)
  calculate = local_namespace['calculate']
  new_df = calculate(df)

  # 獲取和執行策略程式碼
  strategy_str = ai_strategy(new_df, user_msg, add_msg)
  print(strategy_str)
  print("-----------------------")
  exec(strategy_str, globals(), local_namespace)
  AiStrategy = local_namespace['AiStrategy']

  backtest = Backtest(df,
          AiStrategy,
          cash=100000,
          commission=0.004,
          trade_on_close=True,
          exclusive_orders=True,
          )
  stats = backtest.run()
  print(stats)
  return str(stats)


## 5-4 讓 AI 解析回測報告

### 1️⃣2️⃣ 設定 AI 回復內容

In [ ]:
def backtest_analysis(*args):

  content_list = [f"策略{i+1}：{report}"
                  for i, report in enumerate(args)]
  content = "\n".join(content_list)
  content += "\n\n請依資料給我一份約200字的分析報告。若有多個策略, \
                  請選出最好的策略及原因, reply in 繁體中文."

  msg = [{
      "role": "system",
      "content": "你是一位專業的證券分析師, 我會給你交易策略的回測績效,\
                  請幫我進行績效分析.不用詳細講解每個欄位, \
                  重點說明即可, 並回答交易策略的好壞"
  }, {
      "role": "user",
      "content": content
  }]

  reply_data = get_reply(msg)
  return reply_data


### 1️⃣3️⃣ 回測結果分析

In [ ]:
stats = ai_backtest(stock_id="2330.TW",
           period="5y",
           user_msg="MACD",
           add_msg="請設置10%的停損點與20%的停利點")
reply = backtest_analysis(stats)
print(reply)


[*********************100%%**********************]  1 of 1 completed
class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, self.data.Signal_Line):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif crossover(self.data.Signal_Line, self.data.MACD):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2018-11-05 00:00:00
End                       2023-11-03 00:00:00
Duration                   1824 days 00:00:00
Exposure Time [%]                   96.952224
Equity Final [$]                   100257.416
Equity Peak [$]                    100358.616
Return [%]                           0.257416
Buy & Hold Return [%]              133.617021
Return (Ann.) [%]                     0.05338
Volatility (Ann.) [%]                0.117346
Sharpe Ratio          

### 1️⃣4️⃣ 比較多個策略

In [ ]:
# 策略1:MACD+停利停損
stats1 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="MACD",
            add_msg="請設置10%的停損點與20%的停利點")
# 策略2:SMA
stats2 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="SMA",
            add_msg="無")
# 策略3:RSI+停利停損
stats3 = ai_backtest(stock_id="2330.TW", period="5y",
            user_msg="RSI",
            add_msg="請設置10%的停損點與20%的停利點")

reply = backtest_analysis(stats1, stats2, stats3)
print(reply)

[*********************100%%**********************]  1 of 1 completed
class AiStrategy(Strategy):
  def init(self):
    super().init()

  def next(self):
    if crossover(self.data.MACD, self.data.Signal):
        self.buy(size=1,
            sl=self.data.Close[-1] * 0.90,
            tp=self.data.Close[-1] * 1.20)
    elif crossover(self.data.Signal, self.data.MACD):
        self.sell(size=1,
             sl=self.data.Close[-1] * 1.10,
             tp=self.data.Close[-1] * 0.80)
-----------------------
Start                     2018-11-07 00:00:00
End                       2023-11-07 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   97.034596
Equity Final [$]                   100248.662
Equity Peak [$]                     100352.87
Return [%]                           0.248662
Buy & Hold Return [%]              136.752137
Return (Ann.) [%]                    0.051566
Volatility (Ann.) [%]                0.117311
Sharpe Ratio                    